In [1]:
import pandas as pd
import re

### Limpieza y Normalización de Nombres Científicos en el Dataset de Especies Faltantes para Integración con Taxonomía EIDOS

In [ ]:
df_taxonomia = pd.read_excel('../EIDOS_taxonomia.xlsx')
df1 = pd.read_excel("Datos IEET/BD_IEET_2015.xlsx")

In [ ]:
# Cargar el archivo CSV extraido de QGIS en el que se han combinado los shapefiles de las especies que faltaban
df2 = pd.read_csv("Datos IEET/Otras_Especies.csv")

# Eliminar las columnas "layer" y "path" si existen
columnas_a_eliminar = ["layer", "path"]
df2 = df2.drop(columns=[col for col in columnas_a_eliminar if col in df2.columns])

# Funcion para limpiar la columna "Nombre" eliminando el numero final
def limpiar_nombre(nombre):
    nombre_limpio = re.sub(r'\d+$', '', str(nombre)).strip()  # Elimina numero final
    if nombre_limpio == "Streptopelia risori":  # Sustituye el nombre especifico
        return "Streptopelia roseogrisea f. risoria"
    return nombre_limpio

# Aplicar la funcion a la columna "Nombre"
if "Nombre" in df2.columns:
    df2["Nombre"] = df2["Nombre"].apply(limpiar_nombre)

# Ver las primeras filas para comprobar que todo esta correcto
df2.head()

,CUADRICULA,Nombre
0,29TNJ84,Bufo bufo
1,29TNJ94,Bufo bufo
2,29TPJ04,Bufo bufo
3,29TNJ73,Bufo bufo
4,29TNJ83,Bufo bufo


### Asignación Automática de ID Taxonómico a Especies del Inventario IEET Mediante Coincidencia Léxica con la Base de Taxonomía EIDOS

In [ ]:
df = pd.concat([df1, df2])
df = df.fillna("")

In [ ]:
# Quitar parentesis y palabras que empiezan por mayuscula y numeros
df_taxonomia['lowered_name'] = df_taxonomia['name'].str.replace(r'[()&]', '', regex=True).apply(
    lambda x: x[0].lower() + ' '.join([w for w in x[1:].split() if w == w.lower() and not w.isdigit()])
    )

def find_best_match(species):
    row_words = species.replace('(', '').replace(')', '').lower().split()
    if len(row_words) < 2:
        return None # No hay suficientes palabras para buscar
    # Filtrar candidatos
    candidates = df_taxonomia[df_taxonomia['lowered_name'].str.contains(' '.join(row_words[:2]))]
    # Si hay una categoría taxonómica, refinar los candidatos
    if len(row_words) > 3 and row_words[2] in ['f.', 'subsp.', 'var.']:
        candidates = candidates[candidates['lowered_name'].str.contains(f' {row_words[3]} ')]
    else:
        candidates = candidates[~candidates['lowered_name'].str.contains(r'\b(?: f\. | subsp\. | var\. )\b', regex=True)]
    if candidates.empty:
        return None
    # Calcular las puntuaciones de coincidencia
    row_words_set = set(row_words)
    candidates['score'] = candidates['lowered_name'].apply(lambda c: 
        0.9 * len(set(c.split()) & row_words_set) / len(row_words) +
        0.1 * len(set(c.split()) & row_words_set) / len(c.split())
    )
    # Obtener el mejor candidato
    best_match = candidates.loc[candidates['score'].idxmax()]
    return best_match['taxonid']

species_id = {species: find_best_match(species) for species in set(df["Nombre"].to_list())}
df['idtaxon'] = df["Nombre"].apply(lambda x: species_id[x])
df.to_excel('IEET_ubicaciones_idtaxon.xlsx', index=False)